In [ ]:
#| default_exp oauth

# OAuth
> Basic scaffolding for handling OAuth

- eval: false
- skip_exec: true

See the [docs page](https://fastht.ml/docs/explains/oauth.html) for an explanation of how to use this.

In [ ]:
#| export
from fasthtml.common import *
from oauthlib.oauth2 import WebApplicationClient
from urllib.parse import urlparse, urlencode, parse_qs, quote, unquote
import secrets, httpx

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
from IPython.display import Markdown

In [ ]:
#| export
class _AppClient(WebApplicationClient):
    id_key = 'sub'
    def __init__(self, client_id, client_secret, code=None, scope=None, **kwargs):
        super().__init__(client_id, code=code, scope=scope, **kwargs)
        self.client_secret = client_secret

In [ ]:
#| export
class GoogleAppClient(_AppClient):
    "A `WebApplicationClient` for Google oauth2"
    base_url = "https://accounts.google.com/o/oauth2/v2/auth"
    token_url = "https://oauth2.googleapis.com/token"
    info_url = "https://openidconnect.googleapis.com/v1/userinfo"
    
    def __init__(self, client_id, client_secret, code=None, scope=None, **kwargs):
        scope_pre = "https://www.googleapis.com/auth/userinfo"
        if not scope: scope=["openid", f"{scope_pre}.email", f"{scope_pre}.profile"]
        super().__init__(client_id, client_secret, code=code, scope=scope, **kwargs)
    
    @classmethod
    def from_file(cls, fname, code=None, scope=None, **kwargs):
        cred = Path(fname).read_json()['web']
        return cls(cred['client_id'], client_secret=cred['client_secret'], code=code, scope=scope, **kwargs)

In [ ]:
#| export
class GitHubAppClient(_AppClient):
    "A `WebApplicationClient` for GitHub oauth2"
    prefix = "https://github.com/login/oauth/"
    base_url = f"{prefix}authorize"
    token_url = f"{prefix}access_token"
    info_url = "https://api.github.com/user"
    id_key = 'id'

    def __init__(self, client_id, client_secret, code=None, scope=None, **kwargs):
        super().__init__(client_id, client_secret, code=code, scope=scope, **kwargs)

In [ ]:
#| export
class HuggingFaceClient(_AppClient):
    "A `WebApplicationClient` for HuggingFace oauth2"
    prefix = "https://huggingface.co/oauth/"
    base_url = f"{prefix}authorize"
    token_url = f"{prefix}token"
    info_url = f"{prefix}userinfo"
    
    def __init__(self, client_id, client_secret, code=None, scope=None, state=None, **kwargs):
        if not scope: scope=["openid","profile"]
        if not state: state=secrets.token_urlsafe(16)
        super().__init__(client_id, client_secret, code=code, scope=scope, state=state, **kwargs)

In [ ]:
#| export
class DiscordAppClient(_AppClient):
    "A `WebApplicationClient` for Discord oauth2"
    base_url = "https://discord.com/oauth2/authorize"
    token_url = "https://discord.com/api/oauth2/token"
    revoke_url = "https://discord.com/api/oauth2/token/revoke"
    id_key = 'id'

    def __init__(self, client_id, client_secret, is_user=False, perms=0, scope=None, **kwargs):
        if not scope: scope="applications.commands applications.commands.permissions.update identify"
        self.integration_type = 1 if is_user else 0
        self.perms = perms
        super().__init__(client_id, client_secret, scope=scope, **kwargs)

    def login_link(self):
        d = dict(response_type='code', client_id=self.client_id,
                 integration_type=self.integration_type, scope=self.scope) #, permissions=self.perms, prompt='consent')
        return f'{self.base_url}?' + urlencode(d)

    def parse_response(self, code):
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        data = dict(grant_type='authorization_code', code=code)#, redirect_uri=self.redirect_uri)
        r = httpx.post(self.token_url, data=data, headers=headers, auth=(self.client_id, self.client_secret))
        r.raise_for_status()
        self.parse_request_body_response(r.text)

In [ ]:
#| export
class Auth0AppClient(_AppClient):
    "A `WebApplicationClient` for Auth0 OAuth2"
    def __init__(self, domain, client_id, client_secret, code=None, scope=None, redirect_uri="", **kwargs):
        self.redirect_uri,self.domain = redirect_uri,domain
        config = self._fetch_openid_config()
        self.base_url,self.token_url,self.info_url = config["authorization_endpoint"],config["token_endpoint"],config["userinfo_endpoint"]
        super().__init__(client_id, client_secret, code=code, scope=scope, redirect_uri=redirect_uri, **kwargs)

    def _fetch_openid_config(self):
        r = httpx.get(f"https://{self.domain}/.well-known/openid-configuration")
        r.raise_for_status()
        return r.json()

    def login_link(self, req):
        d = dict(response_type="code", client_id=self.client_id, scope=self.scope, redirect_uri=redir_url(req, self.redirect_uri))
        return f"{self.base_url}?{urlencode(d)}"

In [ ]:
# cli = GoogleAppClient.from_file('client_secret.json')

In [ ]:
#| export
@patch
def login_link(self:WebApplicationClient, redirect_uri, scope=None, state=None, **kwargs):
    "Get a login link for this client"
    if not scope: scope=self.scope
    if not state: state=getattr(self, 'state', None)
    return self.prepare_request_uri(self.base_url, redirect_uri, scope, state=state, **kwargs)

Generating a login link that sends the user to the OAuth provider is done with `client.login_link()`.

It can sometimes be useful to pass state to the OAuth provider, so that when the user returns you can pick up where they left off. This can be done by passing the `state` parameter.

In [ ]:
from fasthtml.common import *
from fasthtml.jupyter import *

In [ ]:
redir_path = '/redirect'
port = 8000
code_stor = None

In [ ]:
app,rt = fast_app()
server = JupyUvi(app, port=port)

In [ ]:
#| export
def redir_url(request, redir_path, scheme=None):
    "Get the redir url for the host in `request`"
    scheme = 'http' if request.url.hostname in ("localhost", "127.0.0.1") else 'https'
    return f"{scheme}://{request.url.netloc}{redir_path}"

In [ ]:
@rt
def index(request):
    redir = redir_url(request, redir_path)
    return A('login', href=cli.login_link(redir), target='_blank')

In [ ]:
#| export
@patch
def parse_response(self:_AppClient, code, redirect_uri):
    "Get the token from the oauth2 server response"
    payload = dict(code=code, redirect_uri=redirect_uri, client_id=self.client_id,
                   client_secret=self.client_secret, grant_type='authorization_code')
    r = httpx.post(self.token_url, json=payload)
    r.raise_for_status()
    self.parse_request_body_response(r.text)

In [ ]:
#| export
@patch
def get_info(self:_AppClient, token=None):
    "Get the info for authenticated user"
    if not token: token = self.token["access_token"]
    headers = {'Authorization': f'Bearer {token}'}
    return httpx.get(self.info_url, headers=headers).json()

In [ ]:
#| export
@patch
def retr_info(self:_AppClient, code, redirect_uri):
    "Combines `parse_response` and `get_info`"
    self.parse_response(code, redirect_uri)
    return self.get_info()

In [ ]:
@rt(redir_path)
def get(request, code:str):
    redir = redir_url(request, redir_path)
    info = cli.retr_info(code, redir)
    return P(f'Login successful for {info["name"]}!')

In [ ]:
# HTMX()

In [ ]:
server.stop()

In [ ]:
#| export
@patch
def retr_id(self:_AppClient, code, redirect_uri):
    "Call `retr_info` and then return id/subscriber value"
    return self.retr_info(code, redirect_uri)[self.id_key]

After logging in via the provider, the user will be redirected back to the supplied redirect URL. The request to this URL will contain a `code` parameter, which is used to get an access token and fetch the user's profile information. See [the explanation here](https://fastht.ml/docs/explains/oauth.html) for a worked example. You can either:

- Use client.retr_info(code) to get all the profile information, or
- Use client.retr_id(code) to get just the user's ID.

After either of these calls, you can also access the access token (used to revoke access, for example) with `client.token["access_token"]`.

In [ ]:
#| export
http_patterns = (r'^(localhost|127\.0\.0\.1)(:\d+)?$',)
def url_match(url, patterns=http_patterns):
    return any(re.match(pattern, url.netloc.split(':')[0]) for pattern in patterns)

In [ ]:
#| export
class OAuth:
    def __init__(self, app, cli, skip=None, redir_path='/redirect', error_path='/error', logout_path='/logout', login_path='/login', https=True, http_patterns=http_patterns):
        if not skip: skip = [redir_path,error_path,login_path]
        store_attr()
        def before(req, session):
            auth = req.scope['auth'] = session.get('auth')
            if not auth: return self.redir_login(session)
            res = self.check_invalid(req, session, auth)
            if res: return res
        app.before.append(Beforeware(before, skip=skip))

        @app.get(redir_path)
        def redirect(req, session, code:str=None, error:str=None, state:str=None):
            if not code: session['oauth_error']=error; return RedirectResponse(self.error_path, status_code=303)
            scheme = 'http' if url_match(req.url,self.http_patterns) or not self.https else 'https'
            base_url = f"{scheme}://{req.url.netloc}"
            info = AttrDictDefault(cli.retr_info(code, base_url+redir_path))
            ident = info.get(self.cli.id_key)
            if not ident: return self.redir_login(session)
            res = self.get_auth(info, ident, session, state)
            if not res:   return self.redir_login(session)
            req.scope['auth'] = session['auth'] = ident
            return res

        @app.get(logout_path)
        def logout(session):
            session.pop('auth', None)
            return self.logout(session)

    def redir_login(self, session): return RedirectResponse(self.login_path, status_code=303)
    def redir_url(self, req):
        scheme = 'http' if url_match(req.url,self.http_patterns) or not self.https else 'https'
        return redir_url(req, self.redir_path, scheme)

    def login_link(self, req, scope=None, state=None): return self.cli.login_link(self.redir_url(req), scope=scope, state=state)
    def check_invalid(self, req, session, auth): return False
    def logout(self, session): return self.redir_login(session)
    def get_auth(self, info, ident, session, state): raise NotImplementedError()

### Google helpers

In [ ]:
#| export
try:
    from google.oauth2.credentials import Credentials
    from google.auth.transport.requests import Request
except ImportError:
    Request=None
    class Credentials: pass

In [ ]:
#| export
@patch
def update(self:Credentials):
    "Refresh the credentials if they are expired, and return them"
    if self.expired: self.refresh(Request())
    return self

In [ ]:
show_doc(Credentials.update)

---

### Credentials.update

>      Credentials.update ()

*Refresh the credentials if they are expired, and return them*

In [ ]:
#| export
@patch
def save(self:Credentials, fname):
    "Save credentials to `fname`"
    save_pickle(fname, self)

In [ ]:
show_doc(Credentials.save)

---

### Credentials.save

>      Credentials.save (fname)

*Save credentials to `fname`*

In [ ]:
#| export
def load_creds(fname):
    "Load credentials from `fname`"
    return load_pickle(fname).update()

In [ ]:
#| export
@patch
def creds(self:GoogleAppClient):
    "Create `Credentials` from the client, refreshing if needed"
    return Credentials(token=self.access_token, refresh_token=self.refresh_token, 
        token_uri=self.token_url, client_id=self.client_id,
        client_secret=self.client_secret, scopes=self.scope).update()

In [ ]:
show_doc(GoogleAppClient.creds)

---

### GoogleAppClient.creds

>      GoogleAppClient.creds ()

*Create `Credentials` from the client, refreshing if needed*

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()